# 🤝 Human-in-the-Loop: AI에게 일시정지 버튼 달기

## 💡 핵심 아이디어

AI가 중요한 작업을 하기 전에 **"잠깐! 이거 진짜 해도 돼?"** 라고 사람에게 물어보는 시스템

---

## 📖 배울 내용

1. **Interrupt** - AI 실행을 멈추는 방법
2. **3가지 선택** - 승인 / 수정 / 취소
3. **실전 예제** - 검색 도구를 사용하는 챗봇

---

## 🎬 동작 시나리오

```
고객: "iPhone 가격 알려줘"
  ↓
AI: "검색 도구로 'iPhone 가격' 검색하겠습니다"
  ↓
🛑 STOP! 사람에게 확인
  ↓
사람의 선택:
  ✅ 승인 → 그대로 실행
  ✏️ 수정 → 검색어 변경
  ❌ 취소 → 다른 방식으로
```

---

## 1️⃣ 환경 준비

---



In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 설정


# LLM 초기화
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)

print("✅ LLM 준비 완료")

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/jinhohyeon/Library/Python/3.9/lib/python/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/jinhohyeon/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jinhohyeon/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM 준비 완료


## 2️⃣ 검색 도구

In [5]:
from langchain_core.tools import tool
from langchain_tavily.tavily_search import TavilySearch  # Tavily 검색 툴 임포트

tavily_tool = TavilySearch(max_results=3)   # 뉴스 검색 툴
tavily_tool.invoke({"query": "삼성전자 매출"})

{'query': '삼성전자 매출',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://news.samsungsemiconductor.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-2025%EB%85%84-2%EB%B6%84%EA%B8%B0-%EC%8B%A4%EC%A0%81-%EB%B0%9C%ED%91%9C/',
   'title': '삼성전자, 2025년 2분기 실적 발표 - 삼성전자 반도체 뉴스룸',
   'content': '삼성전자가 연결 기준으로 매출 74.6조원, 영업이익 4.7조원의 2025년 2분기 실적을 발표했다. DS부문은 서버용 고부가 메모리 제품과 파운드리 주요 거',
   'score': 0.8652358,
   'raw_content': None},
  {'url': 'https://www.joongang.co.kr/article/25373527',
   'title': '삼성전자, 역대 최대 매출에 영업익 12조원...반도체서 10배 ...',
   'content': '14일 삼성전자는 연결기준 3분기 매출 86조원, 영업이익 12조1000억원의 잠정 실적을 공시했다. 지난해 같은 기간 대비 매출과 영업이익은 각각 8.72%,',
   'score': 0.8588255,
   'raw_content': None},
  {'url': 'https://www.yna.co.kr/view/AKR20251029197000003',
   'title': '[표] 삼성전자 2012∼2025년 분기별 매출·영업이익',
   'content': '2012년, 1분기, 2분기, 3분기, 4분기, 합계. 매출, 45.27, 47.6, 52.18, 56.06, 201.1. 영업이익, 5.69, 6.46, 8.06, 8.84, 29.05',
   'score': 0.8464638,
   'raw_content'

In [6]:

# ✅ 툴 리스트 구성
tool_list = [tavily_tool]

# ✅ LLM에 툴 바인딩 (initialize_agent 불필요)
llm_with_tools = llm.bind_tools(tool_list)

## 3️⃣ 그래프 구성

In [8]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# 상태
class State(TypedDict):
    messages: Annotated[list, add_messages]

print("✅ 그래프 설정 완료")

✅ 그래프 설정 완료


## 4️⃣ 노드 정의

### 노드 1: AI

In [10]:
def ai_node(state: State):
    """AI가 질문에 답하거나 도구 사용 계획"""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

### 노드 2: 🛑 Human-in-the-Loop (핵심!)



In [11]:
def human_approval(state: State) -> Command:
    """
    AI가 도구를 쓰기 전에 사람에게 확인받는 핵심 노드!
    """
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]

    # AI의 계획 보여주기
    print("\n" + "="*60)
    print("🤖 AI의 계획:")
    print(f"   도구: {tool_call['name']}")
    print(f"   검색어: {tool_call['args']['query']}")
    print("="*60)

    # 🛑 여기서 멈춤!
    decision = interrupt("사람의 승인 대기 중...")

    action = decision["action"]

    # 승인
    if action == "approve":
        print("\n✅ 승인 - 도구 실행")
        return Command(goto="tools")

    # 수정
    elif action == "modify":
        new_query = decision["new_query"]
        print(f"\n✏️ 수정 - 새 검색어: {new_query}")

        modified_message = AIMessage(
            content=last_message.content,
            tool_calls=[{
                "id": tool_call["id"],
                "name": tool_call["name"],
                "args": {"query": new_query}
            }]
        )
        return Command(
            goto="tools",
            update={"messages": [modified_message]}
        )

    # 취소: 새 검색어로 처음부터 다시 하되, 툴 사용 여부는 AI가 판단하고
    # 최종 답변까지 즉시 생성해서 반환(중간에 다시 interrupt로 멈추지 않음)
    elif action == "cancel":
        print("\n❌ 취소 - 새 검색어로 다시 시작")

        # 사람에게 새 검색어를 받음
        new_question = input("\n💬 새 검색어를 입력하세요: ")

        # 새로운 HumanMessage
        restart_message = HumanMessage(content=new_question)

        # 1차 호출: 툴 사용 여부(계획) 판단
        first_ai = llm_with_tools.invoke([restart_message])
        msgs = [restart_message, first_ai]

        # 툴 호출이 있으면 직접 실행하고 결과를 연결해 최종 답변 생성
        if hasattr(first_ai, "tool_calls") and first_ai.tool_calls:
            tc = first_ai.tool_calls[0]

            print("\n" + "="*60)
            print("🤖 AI의 계획:")
            print(f"   도구: {tc['name']}")
            print(f"   검색어: {tc['args'].get('query')}")
            print("="*60)

            # 도구 실행
            tool_result = tavily_tool.invoke(tc["args"])
            tool_message = ToolMessage(
                content=tool_result,
                tool_call_id=tc["id"]
            )
            msgs.append(tool_message)

            # 툴 결과까지 포함해 최종 답변 생성
            final_ai = llm_with_tools.invoke(msgs)
            msgs.append(final_ai)
        else:
            # 툴 불필요: 첫 응답이 최종 답변
            final_ai = first_ai

        print(f"\n🤖 새 검색어 기반 최종 답변:\n{final_ai.content}")

        # 이전 이력 + 새 흐름을 합쳐 종료
        return Command(
            goto=END,
            update={"messages": state["messages"] + msgs}
        )


### 노드 3: 도구 실행

In [12]:
def tools_node(state: State):
    """승인된 도구 실행"""
    last_message = state["messages"][-1]
    tool_call = last_message.tool_calls[0]

    # 도구 실행
    result = tavily_tool.invoke(tool_call["args"])
    print(f"\n🔍 검색 완료: {result}")

    tool_message = ToolMessage(
        content=result,
        tool_call_id=tool_call["id"]
    )
    return {"messages": [tool_message]}

### 라우팅

In [13]:
def route_after_ai(state: State):
    """AI 응답 후 경로 결정"""
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "human_approval"

    return END

print("✅ 노드 정의 완료")

✅ 노드 정의 완료


## 5️⃣ 그래프 조립

In [14]:
builder = StateGraph(State)

builder.add_node("ai", ai_node)
builder.add_node("human_approval", human_approval)
builder.add_node("tools", tools_node)

builder.add_edge(START, "ai")
builder.add_conditional_edges("ai", route_after_ai)
builder.add_edge("tools", "ai")

graph = builder.compile(checkpointer=MemorySaver())

print("✅ 그래프 완성!")
print("\n📊 흐름: 질문 → AI → 🛑확인 → 도구 → AI답변")

✅ 그래프 완성!

📊 흐름: 질문 → AI → 🛑확인 → 도구 → AI답변


In [15]:
builder

---

# 🚀 실습

## 실습 1: 질문하고 멈추기

In [17]:
# 세션 설정
config = {"configurable": {"thread_id": "demo-1"}}

# 질문
question = "최신 캐쥬얼 코디에 대해  알려줘"
print(f"👤 질문: {question}\n")

# 실행 (멈출 때까지)
try:
    result = graph.invoke(
        {"messages": [HumanMessage(content=question)]},
        config
    )
except Exception as e:
    # Interrupt 발생은 정상 동작
    pass

print("\n⏸️  그래프가 멈췄습니다!")
print("   다음 셀에서 승인/수정/취소를 선택하세요.")

👤 질문: 최신 캐쥬얼 코디에 대해  알려줘


🤖 AI의 계획:
   도구: tavily_search
   검색어: 최신 캐쥬얼 코디

⏸️  그래프가 멈췄습니다!
   다음 셀에서 승인/수정/취소를 선택하세요.


## 실습 2-A: ✅ 승인하기

In [19]:
# 승인
result = graph.invoke(
    Command(resume={"action": "approve"}),
    config
)

# 최종 답변 확인
final_message = result["messages"][-1]
print(f"\n🤖 최종 답변:\n{final_message.content}")
print("\n✅ 완료!")


🤖 AI의 계획:
   도구: tavily_search
   검색어: 최신 캐쥬얼 코디

✅ 승인 - 도구 실행

🔍 검색 완료: {'query': '최신 캐쥬얼 코디', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://kr.pinterest.com/dalkis1718/%EC%BA%90%EC%A3%BC%EC%96%BC-%EC%BD%94%EB%94%94/', 'title': '22개의 캐주얼 코디 아이디어 | 가을 패션, 패션 스타일, 여성 패션', 'content': '2023. 8. 28. - Pinterest에서 쥬 우님의 보드 "캐주얼 코디"을(를) 팔로우하세요. 가을 패션, 패션 스타일, 여성 패션에 관한 아이디어를 더 확인해 보세요.', 'score': 0.9994429, 'raw_content': None}, {'url': 'https://www.musinsa.com/content/mz/48157', 'title': '캐주얼을 잘 입는 12가지 노하우 - 무신사', 'content': '오버사이즈 핏 셔츠나 가벼운 재킷으로 두루 입을 수 있는 레이어 유니온(LAYER UNION)의 스웨이드 셔츠를 아우터로 걸치고 무심한 멋의 아메리칸 캐주얼 룩을 완성하자.', 'score': 0.9969246, 'raw_content': None}, {'url': 'https://www.instagram.com/p/DR_kRtREkL7/?hl=ko', 'title': '캐주얼 브랜드 16개 - Instagram', 'content': '캐주얼 브랜드 16개 캐주얼 코디가 어려우신 분들, 이제 막 패션에 관심을 가지신 분들 ‼️ 딱 이거만 봐주시면 됩니다 부담없이 언제든 입기 좋은', 'score': 0.99690056, 'raw_content': None}], 'response_time': 0.66, 'request_id': 'c6f2580e-426f-4a

## 실습 2-B: ✏️ 수정하기

새로운 질문으로 다시 시작합니다.

In [21]:
# 새 세션
config2 = {"configurable": {"thread_id": "demo-2"}}

# 질문
question2 = "현재 유행하는 트렌디한 코디를 추천해줘"
print(f"👤 질문: {question2}\n")

# 실행
try:
    graph.invoke(
        {"messages": [HumanMessage(content=question2)]},
        config2
    )
except:
    pass

print("\n⏸️  멈춤! 검색어를 수정해봅시다.")

👤 질문: 현재 유행하는 트렌디한 코디를 추천해줘


⏸️  멈춤! 검색어를 수정해봅시다.


In [22]:
# 검색어 수정
result = graph.invoke(
    Command(resume={
        "action": "modify",
        "new_query": "현재 유행하는 댄디한 코디를 추천해줘!"
    }),
    config2
)

# 결과
final_message = result["messages"][-1]
print(f"\n🤖 최종 답변:\n{final_message.content}")
print("\n✅ 수정된 검색어로 완료!")


🤖 최종 답변:
최신 트렌드를 반영한 코디를 추천해 드리기 위해 검색해 보겠습니다. 어떤 스타일을 선호하시나요? (예: 캐주얼, 포멀, 스트릿 등)

✅ 수정된 검색어로 완료!


## 실습 2-C: ❌ 취소하기

In [24]:
# 새 세션
config3 = {"configurable": {"thread_id": "demo-3"}}

# 질문
question3 = "최신 유행 여름옷 코디를 추천해줘"
print(f"👤 질문: {question3}\n")

# 실행
try:
    graph.invoke(
        {"messages": [HumanMessage(content=question3)]},
        config3
    )
except:
    pass

print("\n⏸️  멈춤! 검색 없이 AI가 직접 답하게 해봅시다.")

👤 질문: 최신 유행 여름옷 코디를 추천해줘


⏸️  멈춤! 검색 없이 AI가 직접 답하게 해봅시다.


In [25]:
# 취소 및 피드백
result = graph.invoke(
    Command(resume={
        "action": "cancel",
    }),
    config3
)

# 결과
final_message = result["messages"][-1]
print(f"\n🤖 최종 답변:\n{final_message.content}")
print("\n✅ 검색 없이 완료!")


🤖 최종 답변:
어떤 스타일을 선호하세요? 예를 들어, 캐주얼, 포멀, 아니면 특별한 스타일을 찾으시나요?

✅ 검색 없이 완료!


---

# 🎓 핵심 정리

## Human-in-the-Loop 3단계

### 1️⃣ Interrupt로 멈추기
```python
decision = interrupt("사람 승인 대기")
```
→ 그래프 실행이 **멈춥니다**

### 2️⃣ 사람이 결정
```python
# 승인
{"action": "approve"}

# 수정
{"action": "modify", "new_query": "..."}

# 취소
{"action": "cancel", "feedback": "..."}
```

### 3️⃣ Command로 재개
```python
graph.invoke(Command(resume={...}), config)
```
→ 그래프가 **다시 실행**

---

## 3가지 선택

| 선택 | 효과 |
|------|------|
| ✅ 승인 | AI 계획대로 실행 |
| ✏️ 수정 | 파라미터 변경 후 실행 |
| ❌ 취소 | 실행 안 하고 다시 AI에게 |

---

## 실전 활용

1. **비용 관리** - 유료 API 호출 전 확인
2. **품질 보증** - 고객 메시지 검토
3. **안전성** - 데이터 삭제 전 승인
4. **학습** - AI 판단 과정 확인

---

## 💡 기억하세요

> **Human-in-the-Loop = AI의 안전벨트**
>
> AI가 자율적으로 작동하되, 중요한 순간에는 사람이 제어합니다.

---